<a href="https://colab.research.google.com/github/antonpetkoff/identifier-suggestion/blob/master/notebooks/00-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import utilities
import os
import shutil
from subprocess import check_output

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
%env WORKSPACE_DIR=/content/gdrive/My Drive/src

# TODO: how can one read an environment variable?!?
%cd '/content/gdrive/My Drive/src'

env: WORKSPACE_DIR=/content/gdrive/My Drive/src
/content/gdrive/My Drive/src


In [0]:
timestamp = check_output(['date', '-u', "+%Y-%m-%dT%H-%M-%S"]).decode('utf-8').strip()

os.environ['PROJECT_DIR'] = os.path.join(
    os.environ['WORKSPACE_DIR'],
    f'identifier-suggestion-{timestamp}',
)

In [5]:
!git clone https://github.com/antonpetkoff/identifier-suggestion.git --depth 1 "${PROJECT_DIR}"

Cloning into '/content/gdrive/My Drive/src/identifier-suggestion-2020-05-05T06-53-45'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 62 (delta 0), reused 38 (delta 0), pack-reused 0
Unpacking objects: 100% (62/62), done.


In [0]:
os.chdir(os.environ['PROJECT_DIR'])

In [7]:
!pwd
!ls -l

/content/gdrive/My Drive/src/identifier-suggestion-2020-05-05T06-53-45
total 53
-rw------- 1 root root 35149 May  5 06:53 LICENSE
drwx------ 2 root root  4096 May  5 06:53 notebooks
-rw------- 1 root root  6632 May  5 06:53 README.md
drwx------ 2 root root  4096 May  5 06:53 requirements
drwx------ 9 root root  4096 May  5 06:53 src


In [8]:
%tensorflow_version 2.x

import tensorflow
print(tensorflow.__version__)

2.2.0-rc3


In [9]:
# Google Colab has standard libraries like numpy, pandas, matplotlib and TF (of course) pre-installed
!pip install -r requirements/colab.txt

     |████████████████████████████████| 1.4MB 9.1MB/s 
     |████████████████████████████████| 4.3MB 36.9MB/s 
     |████████████████████████████████| 102kB 15.4MB/s 
     |████████████████████████████████| 460kB 57.0MB/s 
     |████████████████████████████████| 112kB 59.7MB/s 
     |████████████████████████████████| 102kB 16.3MB/s 
     |████████████████████████████████| 71kB 12.8MB/s 
     |████████████████████████████████| 71kB 12.5MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=40387e722cc4ab5feca79efe61e2088372400cde251b3e7c6e01a05bcd74c6ea
  Stored in directory: /root/.cache/pip/wheels/bc/ed/6c/028dea90d31b359cd2a7c8b0da4db80e41d24a59614154072e
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=71de681307c73db0483f0eb3fd247057c9f47244fc9edc11c800e06a3d4875e5
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for gql

In [10]:
# provide secrets to the project, e.g. access to wandb
shutil.copy(
    os.path.join(os.environ['WORKSPACE_DIR'], 'secrets/.env'),
    os.environ['PROJECT_DIR']
)

'/content/gdrive/My Drive/src/identifier-suggestion-2020-05-05T06-53-45/.env'

In [0]:
# monkey-patch (mock) os.symlink to be a noop, because wandb.save() uses it, but it is not supported by Google Colab Notebooks
os.symlink = lambda *x: print('Executing mocked noop symlink with arguments', x)

In [22]:
from argparse import Namespace
from src.pipelines.baseline import run

params = {
  'file_data_raw': '../data/interim/method-names-rich/elasticsearch.csv',
  'file_model_dir': 'models/saved/baseline/',
  'dir_preprocessed_data': '../data/interim/preprocessed/',
  'max_input_length': 200,
  'max_output_length': 8,
  'input_vocab_size': 20000,
  'input_embedding_dim': 128,
  'output_vocab_size': 6000,
  'output_embedding_dim': 128,
  'latent_dim': 512,
  'learning_rate': 0.0001,
  'epochs': 10,
  'batch_size': 512,
  'random_seed': 1,
}

run(Namespace(**params))

Experiment parameters:  Namespace(batch_size=512, dir_preprocessed_data='../data/interim/preprocessed/', epochs=10, file_data_raw='../data/interim/method-names-rich/elasticsearch.csv', file_model_dir='models/saved/baseline/', input_embedding_dim=128, input_vocab_size=20000, latent_dim=512, learning_rate=0.0001, max_input_length=200, max_output_length=8, output_embedding_dim=128, output_vocab_size=6000, random_seed=1)


Loading preprocessed files...
Loaded input vocabulary.
Loaded output vocabulary.
Loaded preprocessed files.
example_X shape:  (512, 200)
example_Y shape:  (512, 8)
total loss: 3.4512858390808105, epoch 1, batch 5
total loss: 4.5801801681518555, epoch 1, batch 10
total loss: 3.476195812225342, epoch 1, batch 15
total loss: 3.3813846111297607, epoch 1, batch 20
total loss: 3.65224027633667, epoch 1, batch 25


wandb: Network error resolved after 0:00:24.270104, resuming normal operation.


total loss: 3.3728575706481934, epoch 1, batch 30
total loss: 3.361548662185669, epoch 1, batch 35
total loss: 2.7298834323883057, epoch 1, batch 40
total loss: 2.392657995223999, epoch 1, batch 45
total loss: 2.0537097454071045, epoch 1, batch 50
total loss: 2.2095837593078613, epoch 1, batch 55
total loss: 1.9058653116226196, epoch 1, batch 60
total loss: 2.442979574203491, epoch 1, batch 65
total loss: 2.169740915298462, epoch 1, batch 70
total loss: 2.413036584854126, epoch 1, batch 75
total loss: 2.2418856620788574, epoch 1, batch 80
total loss: 2.367259979248047, epoch 1, batch 85
total loss: 2.7842862606048584, epoch 1, batch 90
total loss: 3.197749614715576, epoch 1, batch 95
total loss: 3.0554401874542236, epoch 1, batch 100
total loss: 3.358518123626709, epoch 1, batch 105
total loss: 2.561542272567749, epoch 1, batch 110
total loss: 3.375640392303467, epoch 1, batch 115
total loss: 2.9185967445373535, epoch 1, batch 120
total loss: 2.472261428833008, epoch 1, batch 125
total

In [0]:
# using the %run magic command doesn't work as expected
# for some reason the GPU Colab instance hangs when trying to read a 50 MB .h5 file

# %run src/pipelines/baseline.py \
#   --file_data_raw ../data/interim/method-names-rich/elasticsearch.csv \
#   --file_model_dir models/saved/baseline/ \
#   --dir_preprocessed_data ../data/interim/preprocessed/ \
#   --max_input_length 200 \
#   --max_output_length 8 \
#   --input_vocab_size 20000 \
#   --input_embedding_dim 128 \
#   --output_vocab_size 6000 \
#   --output_embedding_dim 128 \
#   --latent_dim 512 \
#   --learning_rate 0.0001 \
#   --epochs 1 \
#   --batch_size 256 \
#   --random_seed 1

In [20]:
os.symlink(
    '/content/gdrive/My Drive/src/identifier-suggestion-2020-05-05T06-53-45/models/saved/baseline/encoder.h5',
    './reports/wandb/run-20200505_065418-17sjamuy/encoder.h5'
)

Executing mocked noop symlink with arguments ('/content/gdrive/My Drive/src/identifier-suggestion-2020-05-05T06-53-45/models/saved/baseline/encoder.h5', './reports/wandb/run-20200505_065418-17sjamuy/encoder.h5')
